# Tomographic reconstruction in four dimensions 

In [ ]:
import sys
import os
import importlib
import numpy as np
import matplotlib.pyplot as plt
import proplot as pplt
from tqdm import tqdm
from tqdm import trange

import reconstruct as rec

sys.path.append('/Users/46h/Research/')
from accphys.tools import utils
from accphys.tools import plotting as myplt
from accphys.tools.utils import rotation_matrix

In [ ]:
pplt.rc['axes.grid'] = False
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.sequential'] = 'viridis'

savefig_kws = dict(dpi=300)

In [ ]:
labels = ["x", "x'", "y", "y'"]
labels_norm = [r"$x_n$", r"$x'_n$", r"$y_n$", r"$y'_n$"]

## Introduction

A beam of charged particles in an accelerator is characterized by its distribution of particles in phase space — the space of positions $\{x, y, z\}$ and momenta $\{x', y', z'\}$. I'm currently working on a projection in which we're trying to measure the 4D phase space $f(x, x', y, y')$ of a full-intensity beam in the SNS. 

An approximation of the 4D phase space can be obtained from multiple 1D projections. These projections are easy to obtain by sweeping wires across the beam path. By approximation, I mean that we measure the $4 \times 4$ covaraince matrix of the distribution which gives the second-order moments. I've implemented this method in the SNS, but I won't discuss it here.

In addition to these wire-scanners, the final destination of the beam in the SNS — the target — is coated with a material that releases photons when impacted by the beam. An image of the beam on the target is formed collecting this light. The image gives us the 2D projection of the 4D phase space distribution onto the $x$-$y$ plane. The idea that I'll explore in this post is whether this 2D projection can be used to reconstruct the 4D phase space distribution.

A projection of a distribution is a sum along some direction; for example, a 1D projection $f(x)$ of the 2D distribution $f(x, y)$ is

$$
f(x) = \int_{-\infty}^{\infty}{f(x, y)dy}.
$$

There are many cases in which we don't have access to the distribution but do have access to its projections; for example, 1D or 2D projections of a human body can be obtained using X-rays. The question is then whether these projections can be used to reconstruct the distribution. This is called *tomography*.

## 2D reconstruction algorithms

The case of a 2D reconstruction from 1D projections has been studied extensively. I'll go over three of these approaches: filtered back-projection (FBP), algebraic reconstruction (ART), and maximum entropy (MENT). I'll use the following distribution to test the methods.

In [ ]:
X = np.vstack([
    np.random.multivariate_normal(mean=[0.0, 0.0], cov=[[2.0, 0.4], [0.4, 1.]], size=80000),
    np.random.normal(loc=[-2.0, -2.0], size=(20000, 2)),
    np.random.normal(loc=[2.0, 2.0], size=(20000, 2)),
    np.random.normal(loc=[2.0, -2.0], size=(20000, 2)),
    np.random.normal(loc=[-2.0, 2.0], size=(20000, 2)),
])

n_bins = 60
xmax = 7.0
limits = [(-xmax, xmax), (-xmax, xmax)]
bin_area = rec.get_bin_volume(limits, n_bins)

Z, xedges, yedges = np.histogram2d(X[:, 0], X[:, 1], n_bins, limits, density=True)
xcenters = 0.5 * (xedges[:-1] + xedges[1:])
ycenters = 0.5 * (yedges[:-1] + yedges[1:])

fig, ax = pplt.subplots()
ax.pcolormesh(xcenters, ycenters, Z.T, cmap='dusk_r')
ax.format(xlabel="x", ylabel="x'")
plt.show()

We now take multiple 1D projections of the distribution. Assume that we have the ability to rotate the distribution in the $x$-$x'$ plane. Then each projection onto the $x$ axis will be at a different angle relative to the original distribution. The significance of the rotation will be discussed later on.

In [ ]:
n_proj = 15
angles = np.linspace(0.0, np.pi, n_proj, endpoint=False)
transfer_matrices = [rotation_matrix(angle) for angle in angles]

projections = np.zeros((n_bins, n_proj))
for k in range(n_proj):
    M = transfer_matrices[k]
    X_meas = np.apply_along_axis(lambda row: np.matmul(M, row), 1, X)
    projections[:, k], _ = np.histogram(X_meas[:, 0], n_bins, limits[0], density=True)
    
fig, ax = pplt.subplots()
ax.pcolormesh(projections.T, cmap='dusk_r')
ax.format(xlabel='x', ylabel='Projection index')

### Filtered Back-Projection (FPB)

[FBP theory]

In [ ]:
Z_fbp = rec.fbp(projections, angles, keep_positive=True, density=True, limits=limits)

In [ ]:
fig, axes = pplt.subplots(ncols=3, figsize=None)
kws = dict(cmap='dusk_r', shading='auto')
axes[0].pcolormesh(xcenters, ycenters, Z.T, **kws)
axes[1].pcolormesh(xcenters, ycenters, Z_fbp.T, **kws)
axes[2].pcolormesh(xcenters, ycenters, (Z - Z_fbp).T, colorbar=True)
axes.format(xlabel="x", ylabel="x'", suptitle='FBP Reconstruction')
axes[0].set_title('True')
axes[1].set_title('Reconstructed');
axes[2].set_title('Error)')
plt.show()

### Simultaneous Algebraic Reconstruction (SART) 

[ART theory]

In [ ]:
Z_sart = rec.sart(projections, angles, iterations=2, 
                  keep_positive=True, density=True, limits=limits)

In [ ]:
fig, axes = pplt.subplots(ncols=3)
axes[0].pcolormesh(xcenters, ycenters, Z.T, cmap='dusk_r')
axes[1].pcolormesh(xcenters, ycenters, Z_sart.T, cmap='dusk_r')
axes[2].pcolormesh(xcenters, ycenters, (Z - Z_sart).T, colorbar=True)
axes.format(xlabel="x", ylabel="y", suptitle='SART reconstruction')
axes[0].set_title('Original')
axes[1].set_title('Reconstructed')
axes[2].set_title('Error')
plt.show()

### MENT

[MENT theory]

## 4D phase space reconstruction 

Now we come to the problem of reconstructing the 4D phase space. At first glance, this is a much more difficult problem. 

[Follow Hock (2013)].

In [ ]:
from accphys.tools.accphys_utils import V_matrix_4x4_uncoupled
from accphys.tools.beam_analysis import twiss2D

# Create 4D Gaussian distribution.
X = np.random.normal(size=(200000, 4))

# Add some coupling.
a, b, c = 0.3, 0.4, -0.8
d = b * c / a if a != 0. else 0.
V = np.array([[1., 0., a, b],
              [0., 1., c, d],
              [-d, b, 1., 0],
              [c, -a, 0., 1.]])
X = utils.apply(V, X)

# Normalize x-x' and y-y'.
V = V_matrix_4x4_uncoupled(*twiss2D(np.cov(X.T)))
Xn = utils.apply(np.linalg.inv(V), X)

# Plot the 2D projections of the distribution.
limits_kws = dict(zero_center=False, sigma=4.0)

limits = myplt.auto_limits(X, **limits_kws)
axes = myplt.corner(X, figsize=(6, 6), limits=limits, bins=n_bins)
plt.suptitle('Projections of 4D distribution')
plt.show()

limits = myplt.auto_limits(Xn, **limits_kws)
axes = myplt.corner(Xn, figsize=(6, 6), limits=limits, bins=n_bins)
plt.suptitle('Projections of normalized 4D distribution')
plt.show()

Compute the 4D histogram of the coordinates.

In [ ]:
Ztrue, edges = np.histogramdd(Xn, n_bins, limits, density=True)

centers = []
for _edges in edges:
    centers.append(0.5 * (_edges[:-1] + _edges[1:]))
    
bin_volume = rec.get_bin_volume(limits, n_bins)

Simulate the measurements. $S_{ijkl}$ gives the density at point $(x_i, y_j)$, on the screen for phase advances (angle) $\theta_{x_k} = \pi k / K$ and $\theta_{y_l} = \pi l / L$. . Since there is no problem, we can just rotate each phase plane separately, then loop through and compute the histograms.

In [ ]:
K = 15 # number of angles in x dimension
L = 15 # number of angles in y dimension

muxx = muyy = np.linspace(0., np.pi, K, endpoint=False)

xx_list = []
for mux in tqdm(muxx):
    Mx = rotation_matrix(mux)
    xx_list.append(utils.apply(Mx, X[:, :2])[:, 0])
    
yy_list = []
for muy in tqdm(muyy):
    My = rotation_matrix(muy)
    yy_list.append(utils.apply(My, X[:, 2:])[:, 0])
    
S = np.zeros((n_bins, n_bins, K, L))
for k, xx in enumerate(tqdm(xx_list)):
    for l, yy in enumerate(yy_list):
        S[:, :, k, l], _, _ = np.histogram2d(xx, yy, n_bins, (limits[0], limits[2]))

Run the reconstuction algorithm.

In [ ]:
method = 'SART' # {'SART', 'FBP', 'MENT'}
kws = dict()
Z = rec.rec4D(S, muxx, muyy, n_bins, method=method, 
              keep_positive=True, density=True, limits=limits, **kws)

Check that the reconstructed distribution normalized.

In [ ]:
print('min(Z) = {}'.format(np.min(Z)))
print('sum(Z) * bin_volume = {}'.format(np.sum(Z) * bin_volume))
print()
print('min(Ztrue) = {}'.format(np.min(Ztrue)))
print('sum(Ztrue) * bin_volume = {}'.format(np.sum(Ztrue) * bin_volume))

### Accuracy

In [ ]:
fig, axes = pplt.subplots(ncols=4, figsize=(6, 2), spanx=False)

for i in range(4):
    axes[i].plot(centers[i], rec.project(Z, i), color='black', label='Reconstructed')
    axes[i].plot(centers[i], rec.project(Ztrue, i), color='red8', ls='--', label='True')
    axes[i].set_xlabel(labels_norm[i])
axes[0].legend(loc=(0.0, 1.02), framealpha=0.0, ncol=1)
plt.savefig('_output/projections1D.png', **savefig_kws)

In [ ]:
fig, axes = pplt.subplots(nrows=3, ncols=6, figsize=(8, 4), sharex=False, sharey=False)

indices = [(0, 1), (2, 3), (0, 2), (0, 3), (2, 1), (1, 3)]
pcolormesh_kws = dict(cmap='dusk_r')

for col, (i, j) in enumerate(indices):
    axes[0, col].set_title('{}-{}'.format(labels_norm[i], labels_norm[j]))
    Htrue = rec.project(Ztrue, [i, j])
    H = rec.project(Z, [i, j])
    axes[0, col].pcolormesh(centers[i], centers[j], H.T, cmap='dusk_r')
    axes[1, col].pcolormesh(centers[i], centers[j], Htrue.T, cmap='dusk_r')
    axes[2, col].pcolormesh(centers[i], centers[j], (H - Htrue).T)
axes.format(rowlabels=['Reconstructed', 'True', 'Error'], xticks=[], yticks=[])
plt.savefig('_output/projections2D.png', **savefig_kws)

In [ ]:
# def get_projections(Z):
#     """Return list of shape (3, 3)"""
#     projections = [[], [], []]
#     for i in range(3):
#         for j in range(i + 1):
#             projection = project(Z, [j, i + 1])
#             projections[i].append(projection)
#     return projections

# def plot_projections(projections, limits):
#     pcolormesh_kws = dict(cmap='dusk_r', shading='auto')
#     labels = ["x", "x'", "y", "y'"]
#     fig, axes = myplt.pair_grid_nodiag(4, limits=limits)
#     for i in range(3):
#         for j in range(i + 1):
#             ax = axes[i, j]
#             Z = projections[i][j]
#             xlim, ylim = ax.get_xlim(), ax.get_ylim()
#             xx = np.linspace(xlim[0], xlim[1], Z.shape[0])
#             yy = np.linspace(ylim[0], ylim[1], Z.shape[1])
#             ax.pcolormesh(xx, yy, Z.T, **pcolormesh_kws)
#     return axes

# projections = get_projections(F_true)
# axes = plot_projections(projections, limits)
# plt.suptitle('Reconstructed')

# projections_err = get_projections(np.abs(F - F_true))
# axes = plot_projections(projections_err, limits)
# plt.suptitle('abs(error)');

In [ ]:
error = np.sum((Z - Ztrue)**2)
print('SSQ error = {}'.format(error))